This file just processeses the SQuAD Dataset and stores it in a file format. The five files are :

1) context

2) question

3) answer_text

4) answer_start

5) answer_end

Our next aim should be tokenize each and store in memory as pickle files.( Along with creating a vocabulary using the contexts and questions)

In [20]:
import os
import tqdm 
import numpy as np
import json


class Squad_preprocessor():
    def __init__(self,tokenizer,data_directory = "E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\SQuAD"):
        self.data_directory = data_directory
        self.glove_directory = "E:\\Internships_19\\Internship(Summer_19)\\Q&A_Toolkit\\Dataset_analysis\\glove.6B"
        self.train_file = "train_v2.json"
        self.validation_file = "validation_v2.json"
        self.out_prefix = "train"
        self.tokenizer = tokenizer
        self.num_train_examples = 0
        self.context_lengths = None
        self.vocab = {}
        
    def load_data(self,filename = "train_v2.json"):
        full_path = os.path.join(self.data_directory,filename)
        
        with open(full_path) as datafile:
            self.data = json.load(datafile)
            
#         print(len(self.data["data"]))
            
    def break_file(self, prefix, filename = "train_v2.json", count_examples = False):
        self.load_data(filename)
        self.out_prefix = prefix
        
        ##### creating data directories for different parts of the data namely:
        ## 1) context
        ## 2) question
        ## 3) answer_text
        ## 4) answer_start
        ## 5) answer_end
      
        
        ###       the SQuAD dataset has the following layout:
        # "data" ---> "title", "paragraphs" 
        #                            |
        #                            -----> "context" , "qas"
        #                                                 |
        #                                                 -----> "answers", "id", "is_impossible", "question"
        #
        #    ie. one context has several questions and their respective answers     

        
        with open(os.path.join(self.data_directory, self.out_prefix +'.context'), 'w', encoding='utf-8') as context_file, \
             open(os.path.join(self.data_directory, self.out_prefix +'.question'), 'w', encoding='utf-8') as question_file, \
             open(os.path.join(self.data_directory, self.out_prefix + '.answer_text'), 'w', encoding= 'utf-8') as answer_text_file, \
             open(os.path.join(self.data_directory, self.out_prefix + '.answer_start'), 'w', encoding= 'utf-8') as answer_start_file, \
             open(os.path.join(self.data_directory, self.out_prefix + '.answer_end'), 'w', encoding= 'utf-8') as answer_end_file:
                   
                    for article_idx in tqdm.tqdm(range(len(self.data["data"]))):
                        paragraphs = self.data["data"][article_idx]["paragraphs"] ## all the paragraphs in data directory

                        for paragraph_idx in range(len(paragraphs)):
                            context = paragraphs[paragraph_idx]["context"] ## each context in a given paragraph directory
                            context = context.lower()
                            context_tokens = self.tokenizer(context)

                            ## each context has a range of "answers", "id", "is_impossible", "question" 

                            qas = paragraphs[paragraph_idx]["qas"] ##  "qas" referrring to a single "context"

                            for qas_idx in range(len(qas)):  ### disecting the "qas" into "answers", "id", "is_impossible", "question" 
                                question = qas[qas_idx]["question"]  
                                question = question.lower()
                                question_tokens = self.tokenizer(question)

                                ## we select the first answer id from the range of answers we are given for a particular question
                                
                                if(len(qas[qas_idx]["answers"]) == 0 ):
                                    
                                    answer_text_tokens = "<unk>"
                                    word_level_answer_start = -1
                                    word_level_answer_end = -1
                                    
                                    
                                    
                                else:
                                    
                                    answer_id = 0
                                    answer_text = qas[qas_idx]["answers"][answer_id]["text"]
#                                     print(answer_text)
                                    
                                    answer_text = answer_text.lower()
                                    answer_text_tokens = self.tokenizer(answer_text) ## we atke the first option as the answer

                                    char_level_answer_start = qas[qas_idx]["answers"][answer_id]["answer_start"]
                                    word_level_answer_start = len(context[:char_level_answer_start].split())
                                    word_level_answer_end = word_level_answer_start + len(answer_text.split()) - 1


                                context_file.write(' '.join(token for token in context_tokens)+'\n')
                                question_file.write(' '.join(token for token in question_tokens)+'\n')
                                answer_text_file.write(' '.join(token for token in answer_text_tokens)+'\n')
                                answer_start_file.write(str(word_level_answer_start)+ "\n")
                                answer_end_file.write(str(word_level_answer_end) + "\n")

    
    def conduct_preprocess(self):
        self.break_file("train", self.train_file, True)
        self.break_file("validation", self.validation_file, False)
        
            
            
    
    

In [ ]:
from __future__ import absolute_import
from __future__ import division

import random
import re
import time

import numpy as np
from six.moves import xrange

class Batch(object):
    """
    This class contains all the information required to train a batch
    """
    
    def __init__(self,context_word_indexes,context_word_mask,context_word_tokens,question_word_indexes,question_word_mask,question_word_tokens,answer_tokens,answer_start,answer_end):
         """
        Inputs:
        
          {context/question}_word_indexes: Numpy arrays.
            Shape (batch_size, {context_len/question_len}). Contains padding.
          {context/question}_mask: Numpy arrays, same shape as _word_indexes.
            Contains 1s where there is real data, 0s where there is padding.
          {context/question/answer}_tokens: Lists length batch_size, containing lists (unpadded) of tokens (strings)
         
        """
            
            # ans_span: numpy array, shape (batch_size, 2)
            
            self.context_word_indexes = context_word_indexes
            self.context_word_mask = context_word_mask
            self.context_word_tokens = context_word_tokens
            self.question_word_indexes = question_word_indexes
            self.question_word_mask = question_word_mask
            self.question_word_tokens = question_word_tokens
            self.answer_tokens = answer_tokens
            self.answer_start = answer_start
            self.answer_end = answer_end       
            
         

    def generate_batches(word_to_index, context_path, question_path, answer_path, batch_size, max_context_len, max_question_len, discard_long):
        """
        This function returns a generator object that yields batches.
        The last batch in the dataset will be a partial batch.
        Read this to understand generators and the yield keyword in Python: https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do
        
        Inputs:
        
          word_to_index: dictionary mapping word (string) to index (int)
          context_tokens_file, question_tokens_file, answer_tokens_file: paths to {train/validation}.{context/question/answer} data files
          batch_size: integer, specifies how big the batches should be
          max_context_length, max_question_length: max length of context and question respectively
          discard_long: If True, discard any examples that are longer than context_len or question_len.
                        If False, truncate those exmaples instead.
        """
        context_file, question_file, answer_file = open(context_path), open(question_path), open(answer_path)
        batches = []

        while True:
            if len(batches) == 0: # add more batches
                refill_batches(batches, word_to_index, context_file, question_file, answer_file, batch_size, max_context_len, max_question_len, discard_long)
            if len(batches) == 0:
                break

            # Get next batch. These are all lists length batch_size
            (context_word_indexes, context_word_tokens, question_word_indexes, question_word_tokens, answer_start,answer_end, answer_tokens) = batches.pop(0)

            # Pad context_ids and qn_ids
            qn_ids = padded(qn_ids, question_len) # pad questions to length question_len
            context_ids = padded(context_ids, context_len) # pad contexts to length context_len

            # Make qn_ids into a np array and create qn_mask
            qn_ids = np.array(qn_ids) # shape (batch_size, question_len)
            qn_mask = (qn_ids != PAD_ID).astype(np.int32) # shape (batch_size, question_len)

            # Make context_ids into a np array and create context_mask
            context_ids = np.array(context_ids) # shape (batch_size, context_len)
            context_mask = (context_ids != PAD_ID).astype(np.int32) # shape (batch_size, context_len)

            # Make ans_span into a np array
            ans_span = np.array(ans_span) # shape (batch_size, 2)

            # Make into a Batch object
            batch = Batch(context_ids, context_mask, context_tokens, qn_ids, qn_mask, qn_tokens, ans_span, ans_tokens)

            yield batch

        return
    
    def refill_batches(batches, word2id, context_file, qn_file, ans_file, batch_size, context_len, question_len, discard_long):
        """
        Adds more batches into the "batches" list.
        Inputs:
          batches: list to add batches to
          word2id: dictionary mapping word (string) to word id (int)
          context_file, qn_file, ans_file: paths to {train/dev}.{context/question/answer} data files
          batch_size: int. how big to make the batches
          context_len, question_len: max length of context and question respectively
          discard_long: If True, discard any examples that are longer than context_len or question_len.
            If False, truncate those exmaples instead.
        """
        print "Refilling batches..."
        tic = time.time()
        examples = [] # list of (qn_ids, context_ids, ans_span, ans_tokens) triples
        context_line, qn_line, ans_line = context_file.readline(), qn_file.readline(), ans_file.readline() # read the next line from each

        while context_line and qn_line and ans_line: # while you haven't reached the end

            # Convert tokens to word ids
            context_tokens, context_ids = sentence_to_token_ids(context_line, word2id)
            qn_tokens, qn_ids = sentence_to_token_ids(qn_line, word2id)
            ans_span = intstr_to_intlist(ans_line)

            # read the next line from each file
            context_line, qn_line, ans_line = context_file.readline(), qn_file.readline(), ans_file.readline()

            # get ans_tokens from ans_span
            assert len(ans_span) == 2
            if ans_span[1] < ans_span[0]:
                print "Found an ill-formed gold span: start=%i end=%i" % (ans_span[0], ans_span[1])
                continue
            ans_tokens = context_tokens[ans_span[0] : ans_span[1]+1] # list of strings

            # discard or truncate too-long questions
            if len(qn_ids) > question_len:
                if discard_long:
                    continue
                else: # truncate
                    qn_ids = qn_ids[:question_len]

            # discard or truncate too-long contexts
            if len(context_ids) > context_len:
                if discard_long:
                    continue
                else: # truncate
                    context_ids = context_ids[:context_len]

            # add to examples
            examples.append((context_ids, context_tokens, qn_ids, qn_tokens, ans_span, ans_tokens))

            # stop refilling if you have 160 batches
            if len(examples) == batch_size * 160:
                break

        # Once you've either got 160 batches or you've reached end of file:

        # Sort by question length
        # Note: if you sort by context length, then you'll have batches which contain the same context many times (because each context appears several times, with different questions)
        examples = sorted(examples, key=lambda e: len(e[2]))

        # Make into batches and append to the list batches
        for batch_start in xrange(0, len(examples), batch_size):

            # Note: each of these is a list length batch_size of lists of ints (except on last iter when it might be less than batch_size)
            context_ids_batch, context_tokens_batch, qn_ids_batch, qn_tokens_batch, ans_span_batch, ans_tokens_batch = zip(*examples[batch_start:batch_start+batch_size])

            batches.append((context_ids_batch, context_tokens_batch, qn_ids_batch, qn_tokens_batch, ans_span_batch, ans_tokens_batch))

        # shuffle the batches
        random.shuffle(batches)

        toc = time.time()
        print "Refilling batches took %.2f seconds" % (toc-tic)
        return